In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CreditCardApprovalPrediction") \
    .getOrCreate()

In [3]:
sample_raw_directory = "sample/raw"
sample_data = spark.read.parquet(sample_raw_directory)
sample_data.show()

+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+--------------+------+-----------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|MONTHS_BALANCE|STATUS|mode_status|
+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+--------------+------+-----------+
|5111095|          F|           N|              N|           0|        112500.0|         Working|Secondary / sec

In [4]:
def convert_parquet_to_json(spark_df):
    """
    Convert a Spark DataFrame to JSON format and print the result.

    Parameters:
    - spark_df: Spark DataFrame loaded from Parquet.

    Returns:
    - A list of JSON objects (each row as a dictionary).
    """
    # Convert Spark DataFrame to Pandas DataFrame
    pandas_df = spark_df.toPandas()
    
    # Convert Pandas DataFrame to JSON
    json_data = pandas_df.to_json(orient='records')
    
    # Print the JSON data
    print(json_data)
    
    # Return the JSON data
    return json_data

# Example usage:
# Assuming `sample_raw` is your loaded Spark DataFrame
json_output = convert_parquet_to_json(sample_data)
print(json_output)


[{"ID":5111095,"CODE_GENDER":"F","FLAG_OWN_CAR":"N","FLAG_OWN_REALTY":"N","CNT_CHILDREN":0,"AMT_INCOME_TOTAL":112500.0,"NAME_INCOME_TYPE":"Working","NAME_EDUCATION_TYPE":"Secondary \/ secondary special","NAME_FAMILY_STATUS":"Married","NAME_HOUSING_TYPE":"House \/ apartment","DAYS_BIRTH":-10653,"DAYS_EMPLOYED":-3007,"FLAG_MOBIL":1,"FLAG_WORK_PHONE":0,"FLAG_PHONE":1,"FLAG_EMAIL":0,"OCCUPATION_TYPE":"Core staff","CNT_FAM_MEMBERS":2.0,"MONTHS_BALANCE":-6,"STATUS":"C","mode_status":"C"},{"ID":5061564,"CODE_GENDER":"F","FLAG_OWN_CAR":"N","FLAG_OWN_REALTY":"Y","CNT_CHILDREN":0,"AMT_INCOME_TOTAL":112500.0,"NAME_INCOME_TYPE":"Pensioner","NAME_EDUCATION_TYPE":"Secondary \/ secondary special","NAME_FAMILY_STATUS":"Married","NAME_HOUSING_TYPE":"House \/ apartment","DAYS_BIRTH":-18759,"DAYS_EMPLOYED":365243,"FLAG_MOBIL":1,"FLAG_WORK_PHONE":0,"FLAG_PHONE":0,"FLAG_EMAIL":0,"OCCUPATION_TYPE":null,"CNT_FAM_MEMBERS":2.0,"MONTHS_BALANCE":-28,"STATUS":"C","mode_status":"C"}]
[{"ID":5111095,"CODE_GENDER":"

In [3]:
from process import DataProcessor  
sample_processed_directory = "sample/processed"
data_processor = DataProcessor()
sample_processed = data_processor.process(input_df=sample_data,output_dir=sample_processed_directory)
sample_processed.show()

+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+---------------+----------+----------+---------------+---------------+--------------+------+-----------+------------------+------------------+---------------------+---------------------+---------------+--------------------+--------------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|MONTHS_BALANCE|STATUS|mode_status|               AGE|    YEARS_EMPLOYED|INCOME_PER_FAM_MEMBER|CREDIT_HISTORY_LENGTH|RECENT_ACTIVITY|            features|     scaled_features|
+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+------------

In [5]:
import os
print(os.path.exists("rf_model"))  # Should print True if the path exists


True


In [6]:
print(os.getcwd())

c:\Users\MSI\Desktop\bigdata-20241208T104837Z-001\bigdata\Model Development


In [7]:
model_path = "C:/Users/MSI/Desktop/bigdata-20241208T104837Z-001/bigdata/Model_Deployment/rf_model" 
from pyspark.ml.classification import RandomForestClassificationModel
rf_model = RandomForestClassificationModel.load(model_path)
print("RandomForest model loaded successfully!")


RandomForest model loaded successfully!


In [8]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load the test data
test_data_loaded = spark.read.parquet(sample_processed_directory)

# Make predictions using the random forest model
rf_predictions = rf_model.transform(test_data_loaded)

# Show only the predictions (exclude 'label')
rf_predictions.select("prediction").show()


+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
+----------+



In [5]:
%pip install pyarrow     


   ---------------------------------------- 25.1/25.1 MB 208.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pyspark
from pyspark.sql import SparkSession
#from pyspark.ml import PipelineModel
from pyspark.ml.classification import RandomForestClassificationModel
from pyparsing import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.ml.feature import StandardScalerModel
#from  model.process import DataProcessor
import os

class DataProcessor:
    def __init__(self, scaler_model_path="scaler", feature_columns=None):
        """
        Initialize the DataProcessor class.
        :param scaler_model_path: Path to the pre-trained scaler model.
        :param feature_columns: List of feature column names to be used for modeling.
        """
        self.feature_columns = feature_columns if feature_columns is not None else [
            'INCOME_PER_FAM_MEMBER', 'AMT_INCOME_TOTAL', 'YEARS_EMPLOYED', 'AGE', 'CREDIT_HISTORY_LENGTH', 'RECENT_ACTIVITY'
        ]
        self.scaler_model = StandardScalerModel.load(scaler_model_path)

    def process(self, input_df, output_dir):
        """
        This method processes the input data, applies all transformations, 
        and saves the output to the specified directory.
        :param input_df: Spark DataFrame to be processed.
        :param output_dir: Directory to save the processed data.
        :param file_format: File format to save the data (default is "parquet").
        """
        # Ordinal Encoding for specific categorical columns
        categorical_ordinal_columns = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
        
        # Apply StringIndexer for ordinal columns
        indexers = [StringIndexer(inputCol=col, outputCol=col + "_encoded") for col in categorical_ordinal_columns]
        pipeline = Pipeline(stages=indexers)
        indexed_df = pipeline.fit(input_df).transform(input_df)

        # OneHotEncoding for specific categorical columns
        categorical_onehot_columns = ['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE']
        
        # Apply StringIndexer for one-hot encoding
        indexers = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in categorical_onehot_columns]
        indexer_pipeline = Pipeline(stages=indexers)
        indexed_onehot_df = indexer_pipeline.fit(input_df).transform(input_df)

        # Check for distinct values before applying OneHotEncoder
        valid_columns = []
        for col in categorical_onehot_columns:
            distinct_values = indexed_onehot_df.select(col + "_index").distinct().count()
            if distinct_values >= 2:
                valid_columns.append(col)

        # Apply OneHotEncoder only to valid columns
        encoder = OneHotEncoder(inputCols=[col + "_index" for col in valid_columns],
                                outputCols=[col + "_onehot" for col in valid_columns])
        encoded_df = encoder.fit(indexed_onehot_df).transform(indexed_onehot_df)
        
        # Custom feature engineering
        input_df = input_df.withColumn("AGE", F.abs(input_df['DAYS_BIRTH']) / 365)
        input_df = input_df.withColumn("YEARS_EMPLOYED", F.abs(input_df['DAYS_EMPLOYED']) / 365)

        # Drop unnecessary columns
        input_df = input_df.drop('DAYS_BIRTH', 'DAYS_EMPLOYED')

        # Create new features (e.g., 'INCOME_PER_FAM_MEMBER')
        input_df = input_df.withColumn("INCOME_PER_FAM_MEMBER", input_df['AMT_INCOME_TOTAL'] / input_df['CNT_FAM_MEMBERS'])

        # Feature engineering for 'CREDIT_HISTORY_LENGTH' and 'RECENT_ACTIVITY'
        # Convert 'MONTHS_BALANCE' to numeric type (integer)
        input_df = input_df.withColumn("MONTHS_BALANCE", F.col("MONTHS_BALANCE").cast("int"))

        # Now perform the subtraction operation
        credit_history_length = input_df.groupBy('ID').agg(
            (F.max('MONTHS_BALANCE') - F.min('MONTHS_BALANCE')).alias('CREDIT_HISTORY_LENGTH')
        )
        recent_activity_flag = input_df.groupBy('ID').agg(
            (F.expr("max(CASE WHEN MONTHS_BALANCE >= -4 THEN 1 ELSE 0 END)")).alias('RECENT_ACTIVITY')
        )

        # Join these features back into the main dataframe
        input_df = input_df.join(credit_history_length, on="ID", how="left")
        input_df = input_df.join(recent_activity_flag, on="ID", how="left")

        # Select features for modeling
        selected_features = ['INCOME_PER_FAM_MEMBER', 'AMT_INCOME_TOTAL', 'YEARS_EMPLOYED', 'AGE', 'CREDIT_HISTORY_LENGTH', 'RECENT_ACTIVITY']

        # Create a VectorAssembler to combine features into a single vector column
        assembler = VectorAssembler(inputCols=selected_features, outputCol="features")

        # Apply VectorAssembler to prepare features
        feature_df = assembler.transform(input_df)

        # Apply the pre-trained scaler model
        train_data_scaled = self.scaler_model.transform(feature_df)

        # Save the processed data
        train_data_scaled.write.parquet(output_dir, mode='overwrite')
        return train_data_scaled




# Load the trained model from Parquet
def load_model(model_path="rf_model"):
    """
    Load the trained model stored in Parquet format.
    """
    rf_model = RandomForestClassificationModel.load(model_path)
    print("RandomForest model loaded successfully!")
    return rf_model

sample_processed_directory = "Model Development/sample/processed"
sample_raw_directory = "Model Development/sample/raw"


# Process data using DataProcessor class
def preprocess_data(raw_directory= sample_raw_directory,processed_directory= sample_processed_directory):
    
    sample_data = spark.read.parquet(raw_directory)
    data_processor = DataProcessor()
    sample_processed = data_processor.process(input_df=sample_data,output_dir=processed_directory)
    #sample_processed.show()
    return sample_processed

# predict from a given directory, require a parquet format for both data and model
def predict(processed_directory: str, model:pyspark.ml.classification.RandomForestClassificationModel) -> dict:
    """
    Predict using the loaded model.
    :param features: Input features for prediction.
    :param model: The loaded model.
    :return: Prediction result as a dictionary.
    """
    test_data_loaded = spark.read.parquet(processed_directory)

    # Convert input features to a Spark DataFrame
    #feature_df = spark.createDataFrame([features])
    
    # Apply the model
    prediction = model.transform(test_data_loaded)
    prediction_result = prediction.select("prediction").collect()[0][0]
    
    return {"prediction": prediction_result}



In [ ]:
import pandas as pd
dictr={
  "ID": 5111095,
  "CODE_GENDER": "F",
  "FLAG_OWN_CAR": "N",
  "FLAG_OWN_REALTY": "N",
  "CNT_CHILDREN": 0,
  "AMT_INCOME_TOTAL": 112500.0,
  "NAME_INCOME_TYPE": "Working",
  "NAME_EDUCATION_TYPE": "Secondary / secondary special",
  "NAME_FAMILY_STATUS": "Married",
  "NAME_HOUSING_TYPE": "House / apartment",
  "DAYS_BIRTH": -10653,
  "DAYS_EMPLOYED": -3007,
  "FLAG_MOBIL": 1,
  "FLAG_WORK_PHONE": 0,
  "FLAG_PHONE": 1,
  "FLAG_EMAIL": 0,
  "OCCUPATION_TYPE": "Core staff",
  "CNT_FAM_MEMBERS": 2.0,
  "MONTHS_BALANCE": -6,
  "STATUS": "C",
  "mode_status": "C"
}
input_data = pd.DataFrame([dictr])

# Save the input data as a Parquet file
#parquet_path = "/temp_data/input_data.parquet"
input_data.to_parquet("testfolder/raw", engine='pyarrow', index=False)
        
# preprocess data
data_processed= preprocess_data("testfolder/raw","testfolder/processed")
        
# load the model
model= load_model()

        
# # Load the test data
# test_data_loaded = spark.read.parquet(sample_processed_directory)
        
# Make the prediction
rf_predictions = predict("testfolder/processed", model)

RandomForest model loaded successfully!
